In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
def preprocessing(datatable_pd,GeneName=False,POS=False,SAVE=False):
    datatable_pos = datatable_pd['POS']
    genename = datatable_pd['GeneName'].values
    # delete some columns that were not used in cadd paper
    del_cols = ['CHROM','POS','isDerived','AnnoType','ConsScore',
                'ConsDetail','mapAbility20bp','mapAbility35bp',
                'scoreSegDup','isKnownVariant','ESP_AF','ESP_AFR',
                'ESP_EUR','TG_AF','TG_ASN','TG_AMR','TG_AFR','TG_EUR',
                'GeneID','FeatureID','CCDS','GeneName','Exon',
                'Intron']
    datatable_pd = datatable_pd.drop(columns=del_cols)

    # delete columns without a single value
    datatable_pd = datatable_pd.dropna(axis=1,how='all')

    # fill in values recommended by cadd paper
    values = {'GerpRS':0, 'GerpRSpval':1,'EncExp':0,'EncOCC':5,
              'EncOCCombPVal':0,'EncOCDNasePVal':0,'EncOCFairePVal':0,
              'EncOCpolIIPVal':0,'EncOCctcfPVal':0,'EncOCmycPVal':0,
              'EncOCDNaseSig':0,'EncOCFaireSig':0,'EncOCpolIISig':0,
              'EncOCctcfSig':0,'EncOCmycSig':0,'tOverlapMotifs':0,
              'motifDist':0,'TFBS':0,'TFBSPeaksMax':0,'PolyPhenVal':0,
              'SIFTval':0,'TFBSPeaks':0}
    datatable_pd = datatable_pd.fillna(values)
    
    # transform objects to dummies
    categorical_feature_names = \
    datatable_pd.select_dtypes(include=np.object).columns
    categories={} # contains all the levels in those feature columns
    for f in categorical_feature_names:
        datatable_pd[f] = datatable_pd[f].astype('category')
        categories[f] = datatable_pd[f].cat.categories

    dummy_data = pd.get_dummies(datatable_pd,columns=[col for col in
                                                      categorical_feature_names
                                                      if col not in ['INFO']])
    
    # change info column into scalar column
    dummy_data['INFO'] = datatable_pd['INFO'].astype('category').cat.codes
    
    # drop nan values -TODO
    dummy_data_del_all_nan = dummy_data.copy()
    print('Deleted columns that I do not know how to impute:')
    for col in dummy_data.columns:
        null = dummy_data[col].isnull().values.ravel().sum()
        if null > 0:
            print(null,col)
            dummy_data_del_all_nan = dummy_data_del_all_nan.drop(columns=col)
    
    # normalized the numerical values before any processing afterwards
    min_max_scaler = MinMaxScaler()
    dummy_data_scaled = min_max_scaler.fit_transform(dummy_data_del_all_nan)
    dummy_data_scaled = pd.DataFrame(dummy_data_scaled,
                                     columns=dummy_data_del_all_nan.columns)
    
    
    # save the preprocessed data as csv file
#     print(dummy_data_scaled.shape)
#     print(genename.shape)
#     print(dummy_data_scaled.index)
#     raise NotImplementedError
    if GeneName:
        dummy_data_scaled['key'] = genename
    if POS:
        dummy_data_scaled['POS'] = datatable_pos
    if SAVE:
        res_path = os.path.join('data','dummy_no_nan_data.csv')
        dummy_data_scaled.to_csv(res_path,sep='\t',index=False)
        print('Saved to %s'%res_path)
    
    return dummy_data_scaled

In [3]:
# read data
myh7_path = 'data/myh7/mhy7.tsv'
myo5b_path = 'data/myo5b/myo5b_variants_patho_benign_cadd1.3fullannot_v1.xlsx'
myh7 = pd.read_csv(myh7_path,sep='\t')
myo5b_excel = pd.ExcelFile(myo5b_path)
myo5b = myo5b_excel.parse(myo5b_excel.sheet_names[0])
del myo5b_excel

In [4]:
myo5b.head()

,CHROM,POS,ID,REF,ALT,INFO,Type,Length,isTv,isDerived,...,Intron,oAA,nAA,Grantham,PolyPhenCat,PolyPhenVal,SIFTcat,SIFTval,RawScore,PHRED
0,18,47352774,MYO5B:c.5616-2A>G,T,C,Pathogenic,SNV,0,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334537,6.023
1,18,47361716,MYO5B:c.5392C>T,G,A,Pathogenic,SNV,0,False,True,...,NaN,Q,*,NaN,NaN,NaN,NaN,NaN,14.849415,48.000
2,18,47361725,MYO5B:c.5383C>T,G,A,Pathogenic,SNV,0,False,True,...,NaN,R,*,NaN,NaN,NaN,NaN,NaN,15.701726,52.000
3,18,47365526,MYO5B:c.4840C>T,G,A,Pathogenic,SNV,0,False,True,...,NaN,Q,*,NaN,NaN,NaN,NaN,NaN,14.577744,46.000
4,18,47365610,MYO5B:c.4755_4756dupT,C,CCA,Pathogenic,INS,2,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.553096,35.000


In [5]:
myh7.head()

,#Chrom,Pos,Ref,Anc,Alt,Type,Length,isTv,isDerived,AnnoType,...,nAA,Grantham,PolyPhenCat,PolyPhenVal,SIFTcat,SIFTval,RawScore,PHRED,chr_pos,INFO
0,14,23882063,C,C,T,SNV,0,False,True,CodingTranscript,...,*,NaN,NaN,NaN,NaN,NaN,1.880579,15.47,14_23882063,POPULATION
1,14,23882064,T,T,C,SNV,0,False,True,CodingTranscript,...,W,NaN,NaN,NaN,NaN,NaN,1.823769,15.13,14_23882064,PATHOGENIC
2,14,23882082,T,T,C,SNV,0,False,True,Transcript,...,NaN,NaN,NaN,NaN,NaN,NaN,4.428258,24.20,14_23882082,POPULATION
3,14,23882975,C,C,T,SNV,0,False,True,CodingTranscript,...,D,94.0,benign,0.130,deleterious,0.02,3.518627,23.10,14_23882975,POPULATION
4,14,23882976,C,C,A,SNV,0,True,True,CodingTranscript,...,C,159.0,possibly_damaging,0.572,deleterious,0.00,5.615639,26.60,14_23882976,POPULATION


In [45]:
myh7 = myh7.rename(index=str,columns={'#Chrom':'CHROM','Pos':'POS','Ref':'REF','Alt':'ALT'})
myh7 = myh7.drop(['Anc','chr_pos'],axis=1)

In [46]:
myo5b = myo5b.drop(['ID'],axis=1)

In [47]:
myh7.loc[myh7['INFO']=='POPULATION','INFO']='Benign'
myh7.loc[myh7['INFO']=='PATHOGENIC','INFO']='Pathogenic'

In [48]:
set(myh7.columns).symmetric_difference(set(myo5b.columns))

set()

In [51]:
# concatenate the myh7 and myo5b datasets
myh7_myo5b = pd.concat([myh7,myo5b],axis=0)

In [53]:
myh7_myo5b.dtypes

ALT                 object
AnnoType            object
CCDS                object
CDSpos             float64
CHROM                int64
ConsDetail          object
ConsScore            int64
Consequence         object
CpG                float64
Domain              object
Dst2SplType         object
Dst2Splice         float64
ESP_AF             float64
ESP_AFR            float64
ESP_EUR            float64
EncExp             float64
EncH3K27Ac         float64
EncH3K4Me1         float64
EncH3K4Me3         float64
EncNucleo          float64
EncOCC             float64
EncOCCombPVal      float64
EncOCDNasePVal     float64
EncOCDNaseSig      float64
EncOCFairePVal     float64
EncOCFaireSig      float64
EncOCctcfPVal      float64
EncOCctcfSig       float64
EncOCmycPVal       float64
EncOCmycSig        float64
                    ...   
isKnownVariant        bool
isTv                object
mamPhCons          float64
mamPhyloP          float64
mapAbility20bp     float64
mapAbility35bp     float64
m

In [54]:
processed_myh7_myo5b = preprocessing(myh7_myo5b)

Deleted columns that I do not know how to impute:
74 CDSpos
803 Dst2Splice
3 EncH3K27Ac
78 EncNucleo
224 Grantham
57 cDNApos
1080 mirSVR-Aln
1080 mirSVR-E
1080 mirSVR-Score
74 protPos
74 relCDSpos
74 relProtPos
57 relcDNApos


In [55]:
myh7.select_dtypes(exclude=[np.object]).columns

Index(['CHROM', 'POS', 'Length', 'isDerived', 'ConsScore', 'GC', 'CpG',
       'mapAbility20bp', 'mapAbility35bp', 'scoreSegDup', 'priPhCons',
       'mamPhCons', 'verPhCons', 'priPhyloP', 'mamPhyloP', 'verPhyloP',
       'GerpN', 'GerpS', 'GerpRS', 'GerpRSpval', 'bStatistic', 'mutIndex',
       'dnaHelT', 'dnaMGW', 'dnaProT', 'dnaRoll', 'mirSVR-Score', 'mirSVR-E',
       'mirSVR-Aln', 'targetScan', 'fitCons', 'cHmmTssA', 'cHmmTssAFlnk',
       'cHmmTxFlnk', 'cHmmTx', 'cHmmTxWk', 'cHmmEnhG', 'cHmmEnh',
       'cHmmZnfRpts', 'cHmmHet', 'cHmmTssBiv', 'cHmmBivFlnk', 'cHmmEnhBiv',
       'cHmmReprPC', 'cHmmReprPCWk', 'cHmmQuies', 'EncExp', 'EncH3K27Ac',
       'EncH3K4Me1', 'EncH3K4Me3', 'EncNucleo', 'EncOCC', 'EncOCCombPVal',
       'EncOCDNasePVal', 'EncOCFairePVal', 'EncOCpolIIPVal', 'EncOCctcfPVal',
       'EncOCmycPVal', 'EncOCDNaseSig', 'EncOCFaireSig', 'EncOCpolIISig',
       'EncOCctcfSig', 'EncOCmycSig', 'tOverlapMotifs', 'motifDist',
       'motifECount', 'motifEName', 'motifEHIP

In [56]:
# add genename to processed table
processed_myh7_myo5b['genename'] = myh7_myo5b['GeneName'].values

In [70]:
processed_myh7 = processed_myh7_myo5b.loc[processed_myh7_myo5b['genename']=='MYH7']
processed_myo5b = processed_myh7_myo5b.loc[processed_myh7_myo5b['genename']=='MYO5B']

In [71]:
processed_myo5b = processed_myo5b.drop('genename',axis=1) # drop pos
processed_myh7 = processed_myh7.drop('genename',axis=1) # drop pos
processed_myh7_myo5b.to_csv('data/myh7/myh7_myo5b.csv',index=False,sep='\t')

# analyse, not relevant to this file, but I kept the results though..

In [59]:
from lib.read_data import dataset,Datasets

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# feature extractors
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVC
# finetuning
from sklearn.model_selection import GridSearchCV
# validation
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [72]:
def read_data_set(data_table,test_size=0.25,BENCHMARK=False):
    '''
    convert a pandas dataframe data table into Datasets(dataset,dataset)
    '''
    train, test = train_test_split(data_table,test_size=0.25)
    train_x = train[[col for col in train.columns
    if col not in ['INFO','gavin_res']]]
    features = train_x.columns
    train_x = np.array(train_x)
    test_x = np.array(test[[col for col in train.columns
    if col not in ['INFO','gavin_res']]])
    train_y = np.array(train['INFO'],dtype=np.int8)
    test_y = np.array(test['INFO'],dtype=np.int8)

    # # check what columns are in the train Dataset
    # for i in range(0,len(train_x.columns),5):
    #     print(train_x.columns[i:i+5])

    if BENCHMARK:
        return Datasets(train=dataset(train_x,train_y,features),
                        test=dataset(test_x,test_y,features)),\
                        train['gavin_res'],\
                        test['gavin_res']
    return Datasets(train=dataset(train_x,train_y,features),
                    test=dataset(test_x,test_y,features))

def run_display_output(classifier,test,DRAW=False):
    '''
    get confusion matrix and auc score for test dataset
    (optional) draw roc curve
    '''
    pred = classifier.predict(test.values)
    tn, fp, fn, tp = confusion_matrix(test.labels,pred).ravel()#confusion matrix
    print(tn,fp,fn,tp)
    sensitivity = tp/(fn+tp)
    specificity = tn/(fp+tn)
    prods = classifier.predict_proba(test.values)[:,1]
    fpr, tpr, _ = metrics.roc_curve(test.labels, prods)
    score = metrics.auc(fpr,tpr) #auc score
    if DRAW:
        draw_roc_curve(fpr,tpr,score)

    return sensitivity, specificity, score

def display_res_gavin_and_best_model(param_grid,pipeline,mvid,filename=None):
    '''
    use model defined by pipeline to fit mvid Dataset
    gridsearchCV determine the parameters given in param_grid
    (optional) save the model in path given in filename
    '''
    classifier = GridSearchCV(estimator=pipeline,
                              param_grid=param_grid)

    print('Start training...')
    classifier.fit(mvid.train.values,mvid.train.labels)
    print('Model Description:\n',classifier.best_estimator_)
    if filename:
        pickle.dump(classifier,open(filename,'wb'))
        print('Saved model to path:',filename)
    sensitivity,specificity,score = run_display_output(classifier,mvid.test)
    print('>>> best model results: sensitivity: {:.{prec}}\tspecificity: {:.{prec}f}\tauc:{}'.\
    format(sensitivity,specificity,score,prec=3))
    return classifier

def inference(classifier,dataset):
    sensitivity,specificity,score = run_display_output(classifier,dataset.test)
    print('>>> best model results: sensitivity: {:.{prec}}\tspecificity: {:.{prec}f}\tauc:{}'.\
    format(sensitivity,specificity,score,prec=3))

def read_gavin(gavin_res, labels):
    '''
    compare gavin results with labels for a certain subset of data
    '''
    gavin_res = gavin_res.replace('Pathogenic',1)
    gavin_res = gavin_res.replace('Benign',0)
    tn_g, fp_g, fn_g, tp_g = \
    confusion_matrix(labels, gavin_res.astype(np.int8)).ravel()
    sensitivity_g = tp_g/(fn_g+tp_g)
    specificity_g = tn_g/(fp_g+tn_g)
    return sensitivity_g, specificity_g

In [74]:
if __name__=='__main__':

    # read data
    myh7_data = read_data_set(processed_myh7,BENCHMARK=False)
    myo5b_data = read_data_set(processed_myo5b,BENCHMARK=False)
    # print(data.head())
    # raise NotImplementedError # check the dataset loaded
    print('Dataset loaded.',myh7_data.train.values.shape)

# ================model selection==========================================
    # # PCA + LogisticRegression
    # # Parameters
    n_components = np.arange(10,50,10)
    class_weight = ['balanced']#,{1:2,0:1}]
    param_grid_logr = [{'pca__n_components':n_components,
                   'logr__penalty':['l1'],#'l2'],
                   'logr__C':[2],#3,4,5],
                   'logr__class_weight':class_weight}]
    # pipeline
    pipeline_logr = Pipeline(steps=[('pca',PCA()),
                               ('logr',LogisticRegression())])
    # save model
    filename = os.path.join('model')#,'pca_logr_new.sav')
    # display results
    classifier_logr = display_res_gavin_and_best_model(param_grid_logr,
                                     pipeline_logr,
                                     myh7_data)#,
                                     #filename)
    
#     # display gavin results
#     sensitivity_g,specificity_g = read_gavin(test_gavin,myh7_data.test.labels)
#     print('>>> gavin model results: sensitivity: {:.{prec}}\tspecificity: {:.{prec}f}'.\
#     format(sensitivity_g,specificity_g,prec=3))

Dataset loaded. (637, 214)
Start training...
Model Description:
 Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('logr', LogisticRegression(C=2, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])
109 33 22 49
>>> best model results: sensitivity: 0.69	specificity: 0.768	auc:0.7723665939297759


In [75]:
inference(classifier_logr,myo5b_data)

19 31 2 10
>>> best model results: sensitivity: 0.833	specificity: 0.380	auc:0.705
